In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from sklearn.preprocessing import OneHotEncoder
import tensorflow_hub as hub
from tqdm import tqdm

In [3]:
data = pd.read_csv('../data_merged/merged_total.csv') # +- 1.5 minuten om in te laden

C:\Users\buyse\AppData\Local\Temp\ipykernel_6076\1853855737.py:1: DtypeWarning: Columns (24,25,85,106) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data_merged/merged_total.csv') # +- 1.5 minuten om in te laden


### Onnodige kolommen weglaten

In [4]:
data = data[data['account_adres_geografische_regio'] == 'Oost-Vlaanderen']
cols_to_drop = [
    'account_account_id',
    'account_adres_geografische_regio',
    'afspraak_afspraak_id',
    'afspraak_eindtijd',
    'activiteitvereistcontact_activityid_id',
    'activiteitvereistcontact_reqattendee',
    'account_oprichtingsdatum',
    'aangemaaktop',
    'gewijzigddoor',
    'gewijzigdop',
    'viewedon',
    'visitorkey',
    'webcontent',
    'account_activiteitscode_account',
    'account_activiteitscode_activiteitscode',
    'account_activiteitscode_inf_account_inf_activiteitscodeid',
    'activiteitscode_activiteitscode_id',
    'contact_contactpersoon_id',
    'functie_functie_id',
    'contactfunctie_contactpersoon',
    'persoon_persoon_id',
    'campagne_campagne_id',
    'campagne_einddatum',
    'campagne_startdatum',
    'sessie_eind_datum_tijd',
    'sessie_start_datum_tijd',
    'sessie_sessie_id',
    'inschrijving_datum_inschrijving',
    'inschrijving_inschrijving_id',
    'visit_visit_id',
    'visit_aangemaakt_op',
    'visit_gewijzigd_op',
    'pageview_id',
    'account_industriezone_naam_',
    'account_adres_provincie',
    'account_reden_van_status',
    'account_status',
    'account_voka_nr_',
    'account_adres_land',
    'activiteitscode_status',
    'status',
    'redenvanstatus',
    'type',
    'url',
    'operatingsystem',
    'financieledata_gewijzigd_op',
    'persoon_persoonsnr_',
    'persoon_reden_van_status',
    'persoon_web_login',
    'persoon_mail_regio_antwerpen_waasland',
    'persoon_mail_regio_west_vlaanderen',
    'persoon_mail_regio_vlaams_brabant',
    'persoon_mail_regio_mechelen_kempen',
    'persoon_mail_regio_limburg',
    'persoon_mail_regio_brussel_hoofdstedelijk_gewest',
    'persoon_mail_regio_antwerpen_waasland',
    'persoon_mail_regio_oost_vlaanderen',
    'persoon_mail_regio_voka_nationaal',
    'persoon_mail_thema_duurzaamheid',
    'persoon_mail_thema_financieel_fiscaal',
    'persoon_mail_thema_innovatie',
    'persoon_mail_thema_internationaal_ondernemen',
    'persoon_mail_thema_mobiliteit',
    'persoon_mail_thema_omgeving',
    'persoon_mail_thema_sales_marketing_communicatie',
    'persoon_mail_thema_strategie_en_algemeen_management',
    'persoon_mail_thema_talent',
    'persoon_mail_thema_welzijn',
    'persoon_mail_type_bevraging',
    'persoon_mail_type_communities_en_projecten',
    'persoon_mail_type_netwerkevenementen',
    'persoon_mail_type_nieuwsbrieven',
    'persoon_mail_type_opleidingen',
    'persoon_mail_type_persberichten_belangrijke_meldingen',
    'campagne_campagne_nr',
    'campagne_status',
    'campagne_naam_in_email',
    'campagne_url_voka_be',
    'sessie_sessie_nr_',
    'visit_adobe_reader',
    'visit_browser',
    'visit_contact_naam_',
    'visit_containssocialprofile',
    'visit_email_send',
    'visit_ended_on',
    'visit_entry_page',
    'visit_exit_page',
    'visit_ip_address',
    'visit_ip_organization',
    'visit_keywords',
    'visit_ip_longitude',
    'visit_ip_latitude',
    'visit_operating_system',
    'visit_started_on',
    'visit_time',
    'browser',
    'time',
    'visit_referrer',
    'visit_referrer_type',
    'contact_functietitel',
    'visit_ip_status'
]
data.drop(cols_to_drop, axis=1, inplace=True)
data.drop_duplicates(inplace=True)
data.shape

(2055930, 44)

### Adres kolommen samenvoegen waar mogelijk

In [5]:
# voeg account_adres_geografische_subregio, account_adres_plaats samen tot account_adres
data['account_adres'] = data['account_adres_geografische_subregio'] + ', ' + data['account_adres_plaats'].str.lower().str.capitalize()
# remove the (O.-Vl.) from the string
data['account_adres'] = data['account_adres'].apply(lambda x: re.sub(r'\(.*\)', '', x))
data.drop(['account_adres_geografische_subregio', 'account_adres_plaats'], axis=1, inplace=True)

### Type kolommen veranderen naar juiste dtypes

In [6]:
# Convert the 'visit_ip_postcode' column to strings
data['visit_ip_postcode'] = data['visit_ip_postcode'].astype(str)
# Replace non-numeric characters with a hyphen
data['visit_ip_postcode'] = data['visit_ip_postcode'].str.replace('[a-zA-Z]', '-', regex=True)
# Replace the entire string with '-1' if it contains a hyphen
data['visit_ip_postcode'] = data['visit_ip_postcode'].apply(lambda x: '-1' if '-' in x else x)
# Remove whitespaces
data['visit_ip_postcode'] = data['visit_ip_postcode'].str.replace(' ', '')
# Convert the column to 'int64'
data['visit_ip_postcode'] = data['visit_ip_postcode'].astype('int64')

In [7]:
def change_dtype(data, col, dtype_first, dtype_to, options=None):
    try:
        if dtype_first == 'object' and dtype_to == 'float64':
            data[col] = data[col].astype('str')
            data[col] = data[col].apply(lambda x: x.replace('unknown', '-1'))
            data[col] = data[col].apply(lambda x: x.replace(',', '.'))
            data[col] = data[col].astype(dtype_to)
        
        if dtype_first == 'object' and dtype_to =='int64':
            data[col] = data[col].astype('str')
            data[col] = data[col].apply(lambda x: x.replace('unknown', '-1'))
            data[col] = data[col].astype(dtype_to)
        
        if dtype_first == 'object' and dtype_to =='onehot':
            data[col] = data[col].astype('str')
            data[col] = data[col].apply(lambda x: x.replace('unknown', '-1'))
            data.replace({col: {options[0]: 1, options[1]: 0}}, inplace=True)
    except:
        print("Error: kolommen zijn misschien al omgezet naar het gewenste datatype")


In [8]:
change_dtype(data, 'financieledata_fte', 'object', 'float64')
change_dtype(data, 'financieledata_toegevoegde_waarde', 'object', 'float64')
change_dtype(data, 'contact_status', 'object', 'onehot', ['Actief', 'Inactief'])
change_dtype(data, 'inschrijving_facturatie_bedrag', 'object', 'float64')
change_dtype(data, 'visit_ip_postcode', 'object', 'int64')

### Gecleande dataset tijdelijk opslaan

In [9]:
if os.path.exists('../data_merged/merged_total_cleaned.csv'):
    os.remove('../data_merged/merged_total_cleaned.csv')

data.to_csv('../data_merged/merged_total_cleaned.csv', index=False)

### One Hot Encoding

In [10]:
try:
    del data
except:
    print("data not defined")

data = pd.read_csv('../data_merged/merged_total_cleaned.csv')

In [11]:
def one_hot_encode(df, col):    
    encoder = OneHotEncoder(sparse_output=False) # Use sparse=False to get a dense array
    encoded_data = encoder.fit_transform(df[[col]].values.reshape(-1, 1))

    feature_names = encoder.get_feature_names_out(input_features=[col])
    
    one_hot_df = pd.DataFrame(encoded_data, columns=feature_names)
    one_hot_df.columns = one_hot_df.columns.str.lower()
    one_hot_df.columns = one_hot_df.columns.str.replace(' ', '_')
    one_hot_df.columns = one_hot_df.columns.str.replace('&', 'en')
    one_hot_df.columns = one_hot_df.columns.str.replace('.', '_')
    one_hot_df.columns = one_hot_df.columns.str.replace('(', '')
    one_hot_df.columns = one_hot_df.columns.str.replace(')', '')

    return one_hot_df

In [12]:
cat_cols = data.select_dtypes(include=['object']).columns

In [13]:
for col in cat_cols:
    unique_lists = data[col].unique().tolist()
    if len (unique_lists) < 35:
        print(col)
        if len(unique_lists) == 2 :
            print(unique_lists)
            change_dtype(data, col, 'object', 'onehot', unique_lists)
            
        elif len(unique_lists) > 2:
            print(unique_lists)
            df_temp = one_hot_encode(data, col)
            data = pd.concat([data, df_temp], axis=1)
            data.drop(col, axis=1, inplace=True)

account_ondernemingsaard
['Productie & Diensten', 'Diensten', 'Productie']
account_ondernemingstype
['Familiebedrijf', 'Bedrijf', 'Social Profit', 'Multinational', 'Overheid', 'Eenmanszaak', 'Vrije beroepen', 'Werkgeversorganisaties', 'Onderwijs']
account_primaire_activiteit
['Ijzer en staal', 'Telecom & IT', 'Bouw', 'Technologische industrie & diensten', 'Zorg', 'Overige industrie & diensten', 'Chemie, petrochemie', 'Media', 'Grafische industrie en diensten', 'Glas, rubber en kunststof', 'Vrije beroepen', 'Voeding', 'Agrarische & bio-industrie', 'Consultancy', 'Human capital', 'Horeca & toerisme', 'Papier & karton', 'Distributie, logistiek en transport', 'Hout- en meubelindustrie', 'Textiel, kleding en confectie', 'Financiële diensten', 'Energie', 'Detailhandel', 'Vastgoed', 'Automobiel- en Tweewielerindustrie', 'Milieu', 'Groothandel', 'Accountancy & boekhouding', 'Verenigingen en maatschappelijke organisaties', 'Farmacie', 'Onderwijs', 'Verzekering', 'Overheid']
activiteitscode_naam

### Tijdelijk opslaan

In [14]:
if os.path.exists('../data_merged/merged_total_cleaned_encoded.csv'):
    os.remove('../data_merged/merged_total_cleaned_encoded.csv')
    
data.to_csv('../data_merged/merged_total_cleaned_encoded.csv', index=False)

### Embedding

In [15]:
data = pd.read_csv('../data_merged/merged_total_cleaned_encoded.csv')
data.shape

(2055930, 249)

In [16]:
cat_cols = data.select_dtypes(include=['object']).columns

for col in cat_cols:
    unique_lists = data[col].unique().tolist()
    if len (unique_lists) > 10:
        print(col)
        print(unique_lists)
        print()

campagne_naam
['OV-IN-Netwerkevent duaal met Richtpunt Hamme', 'OV-GR- Family Business Happening 2021 Familiebedrijven met ambitie', 'OV-Plato 2022 - After Summer Drink', 'OV-NW- Voka Bilan 2022', 'OV-NW-Voka Vista - Community Event-Grootmeesters van de Groei', 'OV-IO - Techtrip 2023 - Infosessie', 'OV-Techtrip 2022: VS - Austin & Palo Alto', 'OV-Financieel management voor niet-financiëlen: de jaarrekening, boekhoudkundige inzichten & management accounting 2022', 'OV-NW-Rode Loper Selectie-Film Fest Gent-Filmmuziekconcert Korean Composers', 'OV-NW-Rode Loper Selectie-Gent Jazz-Gregory Porter', 'OV-GR-Family Business Happening 2022', 'OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen', 'OV-NW-Rode Loper Selectie-Festival van Vlaanderen-Stroom-Midzomernacht', 'OV-NW-Voka Bilan 2023', 'OV-NW-VokaUpdate-Big Refresh-Voorjaar 2022', 'OV-Transitio - Jouw bedrijf klaar voor verkoop of overdracht', 'OV-NW-Voka Update-Big Refresh-Najaar 2022', 'OV-NW-Voka Visit-Fabriek Logistiek', 'OV-NW-Vaart Club